# Categorizing credit card transactions in python

## Libraries

In [1]:
## Loading libraries 
import pandas as pd
import numpy as np
import os
import re


## Setting the working directory
os.chdir("-- Your directory path here---")
print(os.getcwd())

FileNotFoundError: [WinError 2] The system cannot find the file specified: '-- Your directory path here---'

##  Steps:

1. Gather credit card transactions(CCT) in a csv file
2. Create a store directory for categorizing transactions
3. Match transactions to a category by matching with store directory
4. Obtain weather information for the region each day
5. Structure the CCT data for modeling
6. Create a model to predict likelihood of going grocery shopping

### Step 1

In [2]:
ccStatements = pd.read_csv("CompleteCCStatements.csv", 
                           header=0, index_col=0)
ccStatements.head()
ccStatements.shape

(761, 7)

### Step 2

A store directory has been created by scraping business lists from the web and adding on to it from a custom list of categories.

file: Store Directory.csv

In [3]:
storeDirectory = pd.read_csv("Store directory.csv", header=0, index_col=0)
print(storeDirectory.head())
print(storeDirectory.shape)
storeDirectory.dtypes

                                  StoreName StoreCategory
1                          Fresh St. Market     Groceries
2                                       IGA     Groceries
3  MarketPlace IGA in British Columbia only     Groceries
4                             Buy-Low Foods     Groceries
5                              Budget Foods     Groceries
(255, 2)


StoreName        object
StoreCategory    object
dtype: object

#### Manipulating the store directory to facilitate matching the transdactions to categories

In [4]:
## 1. Some stores have 'Canada' and 'Canadian' in their names and may be shortened to 'Can' or 'Cdn' in the credit card statements

# Create a subset of the store directoy that satisfies the above condition
temp_df = storeDirectory[storeDirectory.StoreName.str.contains('Canada') | storeDirectory.StoreName.str.contains('Canadian')]
print(temp_df.head(10))
# Creating necessary replacements
# using regex=T because a part of the string is being replaced
temp_df['StoreName'] = temp_df['StoreName'].replace({'Canada':'Can',
                                                     'Canadian':'Cdn'}, regex=True) 
print(temp_df['StoreName'])

# Appending the changed subset to the store directory file
storeDirectory = storeDirectory.append(temp_df, ignore_index=True)
storeDirectory.shape

                                             StoreName StoreCategory
55       Foodland some CO-OP stores in Atlantic Canada     Groceries
57   IGA Extra in Alberta, Manitoba, Quebec, some p...     Groceries
141                                       Kmart Canada     Groceries
163                                      Target Canada     Groceries
171                                      Canadian Tire      Shopping
188                           Real Canadian Superstore      Shopping
204                            Canadian Tire Petroleum          Fuel
211                                        Gulf Canada          Fuel
220                                       Petro-Canada          Fuel
223                                       Shell Canada          Fuel
55            Foodland some CO-OP stores in Atlantic Can
57     IGA Extra in Alberta, Manitoba, Quebec, some p...
141                                            Kmart Can
163                                           Target Can
171          

C:\Users\Nalina\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(267, 2)

In [5]:
## 2. Replacing special characters in store names to match categories better
# Creating a subset of store names that have special characters 
temp_df = storeDirectory[storeDirectory.StoreName.str.contains(r'[^a-zA-Z\d\s\,\.\#\$\-\/()]+', regex=True)]
print(temp_df.head(10))
print(temp_df.shape)

# Replacing the special characters
temp_df.StoreName = temp_df.StoreName.replace(r"[^a-zA-Z\d\s\,\.\#\$\-\/()]+", "", regex=True)
print(temp_df.head(50))
print(temp_df.shape)

# Append the modified subset to Store Directory
storeDirectory = storeDirectory.append(temp_df, ignore_index=True)
print(storeDirectory.shape)

                StoreName StoreCategory
8            Shop n' Save     Groceries
11  Nature�s Fare Markets     Groceries
17  Atlantic Cash & Carry     Groceries
22   Les Entrep�ts Presto     Groceries
26          L'Intermarch�     Groceries
32             Maxi & Cie     Groceries
33        NG Cash & Carry     Groceries
39      T & T Supermarket     Groceries
42          Club Entrep�t     Groceries
47          March� Adonis     Groceries
(59, 2)
                                             StoreName StoreCategory
8                                          Shop n Save     Groceries
11                                Natures Fare Markets     Groceries
17                                Atlantic Cash  Carry     Groceries
22                                 Les Entrepts Presto     Groceries
26                                         LIntermarch     Groceries
32                                           Maxi  Cie     Groceries
33                                      NG Cash  Carry     Groceries


C:\Users\Nalina\Anaconda2\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [6]:
## 3. Reading a custom list of stores as a csv file and appending it to the store directory
customList = pd.read_csv("Custom list of stores.csv",header=0)
print(customList.head(10))
print(customList.dtypes)
customList.shape

   StoreName      StoreCategory
0     Amazon  Shopping - Amazon
1   Doordash         Restaurant
2       AMZN  Shopping - Amazon
3  Bulk Barn          Groceries
4  Marshalls           Shopping
5    Netflix      Entertainment
6   Cineplex      Entertainment
7     Disney      Entertainment
8      Chatr              Phone
9     COGECO           Internet
StoreName        object
StoreCategory    object
dtype: object


(59, 2)

In [7]:
# Append custom list of stores to store directory
storeDirectory = storeDirectory.append(customList,ignore_index=True)
print(storeDirectory.shape)

(385, 2)


In [15]:
## Test code for matching store names

# storeString = pd.Series(['Fortino','Amazon','Walmart','Anjappar','Marshalls','Netflix','Chatr','Cogeco','Gap','Toys','Children'])
# print(storeString.dtype)
# storeString = storeString.str.lower()
# patstring = '|'.join(storeString)
# patstring = '(' + patstring + ')'
# print(patstring)

# ccStatements['Description'] = ccStatements['Description'].str.lower()
# print(ccStatements['Description'].head(10))
# extraction = ccStatements['Description'].str.extractall(patstring)

# print(extraction.shape)
# print(extraction.head(10))

object
(fortino|amazon|walmart|anjappar|marshalls|netflix|chatr|cogeco|gap|toys|children)
1                                 annual fee
2                          annual rebate fee
3      wal-mart supercenter#1107waterdown on
4     amzn mktp ca*mk0524rq0 www.amazon.caon
5        fortinos (waterdown 20 waterdown on
6         fortinos (main street) hamilton on
7                    r m*wyse 289-3190885 on
8     payment - thank you / paiement - merci
9     canadiantire.ca/roadside 8887277478 on
10            cogeco connexion burlington on
Name: Description, dtype: object
(258, 1)
                 0
   match          
4  0        amazon
5  0       fortino
6  0       fortino
10 0        cogeco
13 0        amazon
18 0        amazon
   1        amazon
19 0       fortino
20 0      children
23 0       fortino


In [8]:
## 4. Removing speacial characters in Store names and marking escape sequences to facilitate pattern matching
# # trial on temp_df before running on store directory
# temp_df = storeDirectory[storeDirectory.StoreName.str.contains(r"[^a-zA-Z\d\s\,\.\-\/&()\*\'\!]+", regex=True)]
# print(temp_df.head(50))
# temp_df.shape
# temp_df.StoreName = temp_df.StoreName.replace(r"[^a-zA-Z\d\s\,\.\-\/&()\*\'\!]+", "", regex=True)
# print(temp_df.head(50))

storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[^a-zA-Z\d\s\,\.\-\/&()\*\'\!]+", "", regex=True)


## replace with escape sequences for , . * ? ( ) ! ' " - &
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\,]+", "\,", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\.]+", "\.", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\*]+", "\*", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\?]+", "\?", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\(]+", "\(", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\)]+", "\)", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\!]+", "\!", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\']+", "\'", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\-]+", "\-", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\&]+", "\&", regex=True)
storeDirectory.StoreName = storeDirectory.StoreName.replace(r"[\s]+", "\s?", regex=True)
#storeDirectory.StoreName = storeDirectory.StoreName.replace(r'[\"]+', '\"', regex=True)

print(storeDirectory.head(50))

                                            StoreName StoreCategory
0                               Fresh\s?St\.\s?Market     Groceries
1                                                 IGA     Groceries
2   MarketPlace\s?IGA\s?in\s?British\s?Columbia\s?...     Groceries
3                                    Buy\-Low\s?Foods     Groceries
4                                      Budget\s?Foods     Groceries
5                                    Nesters\s?Market     Groceries
6                            Meinhardt\s?Fine\s?Foods     Groceries
7                                     Quality\s?Foods     Groceries
8                                    Shop\s?n'\s?Save     Groceries
9                                          AG\s?Foods     Groceries
10                                  Choices\s?Markets     Groceries
11                           Natures\s?Fare\s?Markets     Groceries
12                          Overwaitea\s?Food\s?Group     Groceries
13                       Bulkley\s?Valley\s?Whol

### Step 3

Matching transactions to category.

In [9]:
storeDirectory.StoreName = storeDirectory.StoreName.str.lower()
ccStatements['Description'] = ccStatements['Description'].str.lower()
patstring = '|'.join(storeDirectory.StoreName)
patstring = '(' + patstring + ')'
print(patstring)

# extraction = (ccStatements['Description'].str.extractall(patstring)
#               .drop_duplicates()
#               .rename(columns={0: 'StoreName'}))

extraction = (ccStatements['Description'].str.extractall(patstring)
              .groupby(level=0)
              .agg(', '.join)
              .rename(columns={0: 'StoreName'}))

print(extraction.shape)
print(extraction.head(10))
#extraction.to_csv("Extraction_matches.csv",header=True)

(fresh\s?st\.\s?market|iga|marketplace\s?iga\s?in\s?british\s?columbia\s?only|buy\-low\s?foods|budget\s?foods|nesters\s?market|meinhardt\s?fine\s?foods|quality\s?foods|shop\s?n'\s?save|ag\s?foods|choices\s?markets|natures\s?fare\s?markets|overwaitea\s?food\s?group|bulkley\s?valley\s?wholesale|pricesmart\s?foods|save\-on\-foods|urban\s?fare|atlantic\s?cash\s?\&\s?carry|atlantic\s?superstore|axep|bloor\s?street\s?market|dominion|les\s?entrepts\s?presto|extra\s?foods|fortinos|freshmart|l'intermarch|loblaws|loblaw\s?greatfood|loblaws\s?citymarket|lucky\s?dollar\s?foods|maxi|maxi\s?\&\s?cie|ng\s?cash\s?\&\s?carry|no\s?frills|provigo|shop\s?easy\s?foods|pharmaprix|supervalu|t\s?\&\s?t\s?supermarket|valu\-mart|wholesale\s?club|club\s?entrept|your\s?independent\s?grocer|independent\s?citymarket|zehrs\s?markets|les\s?5\s?saisons|march\s?adonis|march\s?ami|march\s?extra|march\s?richelieu|super\s?c|lawtons|needs\s?convenience|foodland\s?some\s?co\-op\s?stores\s?in\s?atlantic\s?canada|freshco|iga\

In [10]:
# Add Category column from store directory df to extract df
ccStatements_matched = pd.concat([ccStatements,extraction], axis=1)
#.reset_index(drop=True)
print(ccStatements_matched.shape)
print(ccStatements_matched.head(10))

# ccStatements_matched.to_csv("CC Statements Matched.csv",header=True)

(761, 8)
   Transaction Date Posting Date                             Description  \
1             1-Feb        3-Feb                              annual fee   
2             3-Feb        3-Feb                       annual rebate fee   
3            11-Oct       15-Oct   wal-mart supercenter#1107waterdown on   
4            15-Oct       15-Oct  amzn mktp ca*mk0524rq0 www.amazon.caon   
5            16-Oct       19-Oct     fortinos (waterdown 20 waterdown on   
6            16-Oct       19-Oct      fortinos (main street) hamilton on   
7            17-Oct       19-Oct                 r m*wyse 289-3190885 on   
8            17-Oct       19-Oct  payment - thank you / paiement - merci   
9            17-Oct       19-Oct  canadiantire.ca/roadside 8887277478 on   
10           20-Oct       21-Oct          cogeco connexion burlington on   

    Amount             Category Month  Year         StoreName  
1       39          Banking fee   Feb  2020               fee  
2      -39          Bankin

In [25]:
matches = ccStatements_matched['StoreName']
print(matches.shape)

matches = matches.str.split(',',expand=True)
print(matches.shape)

matches = matches.rename({0:'Match1', 1:'Match2', 2:'Match3'}, axis=1)
print(matches.head(20))

storeDirectory['StoreName'] = storeDirectory['StoreName'].str.lower()
storeDirectory['StoreName'] = storeDirectory['StoreName'].str.strip()

# Matching store Category for Column 1
matches.Match1 = matches.Match1.replace(r"[\-]+", "\-", regex=True)
matches.Match1 = matches.Match1.replace(r"[\s]+", "\s?", regex=True)
matches['Match1'] = matches['Match1'].str.lower()
matches['Match1'] = matches['Match1'].str.strip()
matches = pd.merge(matches,storeDirectory, 
                   left_on='Match1',
                  right_on='StoreName',
                  how="left")
print(matches.shape)
print(matches.head(30))
print("\n\nNon missing counts in this df:")
print(matches.count())

matches = matches.drop(['Match1','Match2','Match3','StoreName'], axis=1)
print(matches.shape)

# Adding the column StoreCategory to the df ccStatements_matched
ccStatements_matched1 = pd.concat([ccStatements,matches], axis=1)
print(ccStatements_matched1.shape)
print(ccStatements_matched1.head(50))

# # Exporting ccStatements_matched1 to csv
# ccStatements_matched1.to_csv("CC statements macthed with storeCategory left.csv",header=True)

(761L,)
(761, 3)
            Match1   Match2 Match3
1              fee     None   None
2              fee     None   None
3         wal-mart   superc   None
4             amzn   amazon   None
5         fortinos     None   None
6         fortinos     None   None
7             wyse     None   None
8          payment     None   None
9     canadiantire     None   None
10          cogeco     None   None
11       no frills     None   None
12             NaN      NaN    NaN
13            amzn   amazon   None
14     tim hortons     None   None
15  indian grocers     None   None
16         wingery     None   None
17        wal-mart   superc   None
18          amazon   amazon   None
19        fortinos     None   None
20             NaN      NaN    NaN
(762, 5)
              Match1   Match2 Match3         StoreName        StoreCategory
0                fee     None   None               fee          Banking fee
1                fee     None   None               fee          Banking fee
2          